In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
#from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn import set_config
set_config(display="diagram")
from sklearn.naive_bayes import ComplementNB
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, roc_auc_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report
#from sklearn.metrics import roc_curve, auc,roc_curve,roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
#import xgboost as xgb
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv('/Users/mahmoud/Car_Price_Prediction/V3/working_dataset.csv')

In [3]:
df = df.drop(columns=['Unnamed: 0'])
df

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Badge Label,Car Rating,Car Reviews,Accidents or damage,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
0,2016 Audi A4 2.0T Premium,92124.0,NJ State Auto Used Cars,4.200000,672.0,14359.0,Good Deal,4.2,"['April 22, 2021\nBy Tommy from San Antonio tx...",None reported,...,All-wheel Drive,21.0,E85 Flex Fuel,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo Flexible Fuel,WAUBFAFLXGN009815,2016,Audi,A4,2.0T Premium
1,2015 Buick Enclave Leather,111488.0,NJ State Auto Used Cars,4.200000,672.0,13517.0,Fair Deal,4.2,"['March 26, 2021\nBy Leo Appreciates a great r...",At least 1 accident or damage reported,...,Front-wheel Drive,17.0,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,5GAKRBKD6FJ125566,2015,Buick,Enclave,Leather
2,2012 Jeep Grand Cherokee Laredo,133821.0,Road Masters II Inc.,3.849628,77.0,9995.0,Great Deal,4.5,"['June 28, 2023\nBy CatTayl from Chatsworth, I...",None reported,...,Four-wheel Drive,16.0,E85 Flex Fuel,Automatic,"3.6L V-6 DOHC, variable valve control, engine ...",1C4RJFAGXCC127988,2012,Jeep,Grand,Cherokee Laredo
3,2010 Toyota Camry LE,116643.0,Arrow Motors,4.800000,220.0,9999.0,Good Deal,5.0,"['January 24, 2023\nBy Pauline from St bernard...",At least 1 accident or damage reported,...,Front-wheel Drive,22.0,Gasoline,6-Speed Automatic,2.5L I4 16V MPFI DOHC,4T1BF3EK9AU561578,2010,Toyota,Camry,LE
4,2016 Mazda Mazda6 i Touring,111588.0,Arrow Motors,4.800000,220.0,13499.0,Good Deal,5.0,"['October 30, 2022\nBy RChico456 from Las Vega...",At least 1 accident or damage reported,...,Front-wheel Drive,25.0,Gasoline,6-Speed Manual,2.5L I4 16V GDI DOHC,JM1GJ1V52G1488572,2016,Mazda,Mazda6,i Touring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9797,2021 Mitsubishi Mirage ES,15035.0,CarVision of Maple Shade,3.849628,0.0,13999.0,Fair Deal,4.9,"['April 18, 2021\nBy LackeyMO from Neosho, Mis...",At least 1 accident or damage reported,...,Front-wheel Drive,33.0,Gasoline,5-Speed Manual,1.2L I3 12V MPFI DOHC,ML32AUHJ7MH010688,2021,Mitsubishi,Mirage,ES
9798,2006 Toyota Corolla S,154352.0,Bethlehem Ford,3.849628,0.0,6524.0,Great Deal,5.0,"['April 27, 2023\nBy Kc from Whittier, MA\nUse...",At least 1 accident or damage reported,...,Front-wheel Drive,30.0,Gasoline,4-Speed Automatic,1.8L I4 16V MPFI DOHC,2T1BR32E16C696228,2006,Toyota,Corolla,S
9799,2008 Suzuki SX4 Base (A4),46460.0,Napoli Kia,3.849628,0.0,8924.0,Great Deal,4.9,"['July 2, 2023\nBy Lee from Dayton ohio \nOwn...",None reported,...,All-wheel Drive,21.0,Gasoline,4-Speed Automatic,2.0L I4 16V MPFI DOHC,JS2YB413385100353,2008,Suzuki,SX4,Base (A4)
9800,2014 Hyundai Santa Fe Sport 2.0L Turbo,152296.0,Pine Belt Chevy of Lakewood,3.849628,0.0,8643.0,Good Deal,5.0,"['June 10, 2021\nBy Adelaide from NC\nOwns thi...",At least 1 accident or damage reported,...,Front-wheel Drive,19.0,Gasoline,6-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,5XYZU3LA5EG187545,2014,Hyundai,Santa,Fe Sport 2.0L Turbo


In [7]:
mean_prices = df.groupby('Badge Label')['Price'].mean().reset_index()

mean_prices['Price'] = mean_prices['Price'].apply(lambda x: round(x, 2))

bar_chart = go.Figure()

bar_chart.add_trace(
    go.Bar(
        x=mean_prices['Badge Label'],
        y=mean_prices['Price'],
        text=mean_prices['Price'],
        textposition='auto',  
    )
)

bar_chart.update_layout(
    title='Average Price by Deal',
    xaxis_title='Type of Deal',
    yaxis_title='Average Price',
)

bar_chart.show()


In [13]:
df.columns

Index(['Car Name', 'Mileage', 'Dealer Name', 'Dealer Rating', 'Review Count',
       'Price', 'Badge Label', 'Car Rating', 'Car Reviews',
       'Accidents or damage', '1-owner vehicle', 'Personal use only',
       'Open recall', 'Exterior color', 'Interior color', 'Drivetrain', 'MPG',
       'Fuel type', 'Transmission', 'Engine', 'VIN', 'Year', 'Make', 'Model',
       'Type'],
      dtype='object')